In [ ]:
# Test performance of clustering algorithms while allowing for outliers and overlapping communities.

# Ego-split + CAS
# ECG + CAS
# HSLC (edge version)

# compare with F1s 1/(1/F1(labels, predict) + 1/F1(predict, labels))